# Training Perceptron with Genetic Algorithm
# Introduction
The problem:
In this assignment, we’ll be using both a Genetic Algorithm and Perceptron to perform binary classification on a breast cancer dataset. We will do the classification by finding a hyperplane which separates the data points of the two different classes, and we will compare the results of these two approaches. 

We have provided code stubs in this notebook to get you started, and give hints about the structure of the code.

You need to compare the performance of 3 algorithms on the dataset:
- Genetic Algorithm with variations as described later
- Perceptron Algorithm which you will code
- SciPy implementation of Perceptron Algorithm

You will need to submit a report along with the finished Jupyter notebook in which you should report the performances in the form of tables as well as plots.

![picture](https://drive.google.com/uc?export=view&id=1wRUX0I803AvXbbkWiD1r4_MKuE3mBQdr)

## Approach:  
Let’s start with a few definitions.
- Individual (aka “chromosome”): a plane in space. It is specified by a weight vector and a bias.
- Population: a collection of possible planes (i.e., collection of individuals)
- Parents: two planes that are combined to create a new plane
- Mating pool: a collection of parents that are used to create our next population (thus creating the next generation of plane)
- Fitness: a function that tells us how good each plane is (in our case, how effectively the plane separates the dataset)
- Mutation: a way to introduce variation in our population by randomly modifying values of the plane's coefficients
- Elitism: a way to carry the best individuals into the next generation    
  
Our GA will proceed in the following steps:
- 1. Create the population
- 2. Rank the population according to fitness in decreasing order
- 3. Select the mating pool
- 4. Breed
- 5. Mutate to create the next generation
- 6. Repeat  
Now, let’s see this in action.

In [ ]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
import operator
import math
from sklearn.datasets import load_breast_cancer
import sklearn
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
maxweight = 20.0 # This constant will be used to generate random weights and biases for planes

Here we load the dataset. It is a `(N, dim)` matrix containing a set of points in 30-dimensional space

In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()

dataset = breast_cancer.data
print(dataset.shape)
dim = dataset.shape[1] # Dimension of each sample in the dataset

targets = breast_cancer.target
print(targets.shape)

We separate the data into X_train, X_test, Y_train, and Y_test using `train_test_split`.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset, targets, test_size = 0.1, stratify = targets)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

## Create necessary classes and functions
We first create a Plane class that will allow us to create and handle our planes. Again, these are represented by the w vector, and the b value. Within the Plane class, there is a predict function, which generates a prediction based on this plane and a given point. Predictset allows for predictions for a set of points.

We have two fitness functions. One of them finds the number of points that are correctly classified with this Plane. The other one should take the inverse of the distance of the minimized misclassified vector. In both cases, a larger fitness score is better. 

The accuracy function will run predictions and then determine the accuracy of those predictions. 

In [ ]:
class Plane:
    def __init__(self, w, b):
        """
        w : Weight vector of size (dim,)
        b : Bias (float)
        """
        self.w = w
        self.b = b
    
    def __repr__(self):
        return ' '.join([str(x) for x in np.append(self.w, self.b)])
    
    def predict(self, point):
        """
        point : [x, y]
        """
        if np.dot(self.w, np.array(point)) + b > 0:
            return 1
        else:
            return 0
        
    def predictset(self, points):
        """
        points: Matrix containing set of points of size (n, dim)
        This function returns predictions for multiple examples
        """
        return ((points @ self.w + self.b) > 0).astype(int)
    
    def fitness_numcorrect(self, data, targets):
        """
        data : n x dim matrix
        targets : n x 1 matrix of 0's and 1's representing the ground truth classification
        Our first fitness function is defined to be the number of correctly classified points in the dataset
        """
        predictions = self.predictset(data)
        return np.sum((predictions == targets).astype(int))

    def your_own_fitness(self, data, targets):
        """
        This is the 2nd fitness function which you should implement from the 
        lecture slides (Lecture 3, Rosenblatt's Algorithm)
        """
        # YOUR CODE HERE
        predictions = self.predictset(data)
        misclassified = []
        for i in targets:
            if i == 0:
                misclassified.append(-1)
            else:
                misclassified.append(1)
        
        d = np.multiply((data @ self.w + self.b), misclassified)[predictions != targets]
        sum_misclassified = -1/(np.sum(d))
        return sum_misclassified
                                  
    def fitness(self, data, targets, fitness_type):
        """
        Wrapper function for fitness.
        fitness_type is a string specifying which fitness function to use.
        """
        if fitness_type == "numcorrect":
            return self.fitness_numcorrect(data, targets)
        elif fitness_type == "your_own_fitness":
            return self.your_own_fitness(data, targets)

        print("Invalid fitness type")

    def accuracy(self, data, targets):
        """
        data : n x dim matrix
        targets : n x 1 matrix of 0's and 1's representing the ground truth classification
        This function should return the accuracy of the plane on the dataset
        """
        predictions = self.predictset(data)
        return float(np.sum((predictions == targets).astype(int))) / len(data)
      

## Create our initial population
Plane generator.  
We now can make our initial population (aka first generation). To do so, we need a way to create a function that produces random planes. To create an individual, we randomly select the weights of the plane equation. Even though we are starting out with a completely random initial population, there is still a chance for convergence. 

The first function here produces one random individual, and in the next function, we create the whole initial population by repeatedly calling randomPlane().

In [ ]:
def randomPlane():
    """
    Creates a random hyperplane. The components are uniformly selected at random
    from -maxweight to maxweight
    """
    w = maxweight * (np.random.rand(dim) - 0.5)
    b = maxweight * np.random.rand() - maxweight/2

    return Plane(w=w, b=b)

def initialPopulation(popSize):
    """
    Create inital population of a given size.
    Returns a list of random planes
    """
    pop = []
    for i in range(popSize):
        temp = randomPlane()
        pop.append(temp)

    return pop

Note: we only have to use these functions to create the initial population. Subsequent generations will be produced through breeding and mutation.

## Create the genetic algorithm - Rank by Fitness
Rank individuals  
Next, the evolutionary fun begins. To simulate our “survival of the fittest”, we can make use of Fitness to rank each individual in the population. Our output will be an ordered list with the plane IDs and each associated fitness score.

In [ ]:
def rankPopulation(population, fitness_type="numcorrect"):
    """
    This function sorts the given population in decreasing order of the fitness score.
    Returns a list of ordered pairs of (index, fitness) where index is the index of the
    individual within the population
    """
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = population[i].fitness(X_train, Y_train, fitness_type)
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

## Select the mating pool
There are a few options for how to select the parents that will be used to create the next generation. The most common approaches are either fitness proportionate selection (aka “roulette wheel selection”) or tournament selection:
- Fitness proportionate selection (the version implemented below): The fitness of each individual relative to the population is used to assign a probability of selection. Think of this as the fitness-weighted probability of being selected.
- Tournament selection: A set number of individuals are randomly selected from the population and the one with the highest fitness in the group is chosen as the first parent. This is repeated to chose the second parent.  

Another design feature to consider is the use of elitism. With elitism, the best performing individuals from the population will automatically carry over to the next generation, ensuring that the most successful individuals persist.  
For the purpose of clarity, we’ll create the mating pool in two steps. First, we’ll use the output from rankPopulation to determine which planes to select in our selection function. Then, we set up the roulette wheel by calculating a relative fitness weight for each individual. Next, we compare a randomly drawn number to these weights to select our mating pool. We’ll also want to hold on to our best planes, so we introduce elitism. Ultimately, the selection function returns a list of plane IDs, which we can use to create the mating pool in the matingPool function.

In [ ]:
def selection(popRanked, eliteSize):
    """
    popRanked: output of rankpopulation()
    eliteSize: number of highest ranked individuals we will retain in the next generation
    This function returns a list of indices of individuals selected to form the mating pool
    """
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

Now that we have the IDs of the planes that will make up our mating pool from the selection function, we can create the mating pool. We’re simply extracting the selected individuals from our population.

In [ ]:
def matingPool(population, selectionResults):
    """
    Create the mating pool
    """
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

## Breed
With our mating pool created, we can create the next generation in a process called crossover (aka “breeding”). Each plane can be represented by a combined list, containing the w vector followed by the b constant. 

In crossover_ordered, we randomly select a subset of the first parent string and then fill the remainder of the plane with the genes from the second parent in the order in which they appear, without duplicating any genes in the selected subset from the first parent. See the image below for an example. 

There is a crossover wrapper function, which is what gets called later on. Inside of this function, based on the passed crossover type, we call the appopriate function to be used in the genetic algorithm.

In addition to ordered crossover, you need to implement two additional crossover functions which you must devise.

![picture](https://drive.google.com/uc?export=view&id=1EnVVFtafYwaTaNkyFTKHTlxV49ufCwfB)

In [ ]:
def crossover_ordered(plane1, plane2):
    """
    plane1, plane2 : type Plane
    Ordered crossover as described in the write-up
    """

    p1 = np.append(plane1.w, plane1.b)
    p2 = np.append(plane2.w, plane2.b)
    crossover_start = random.randint(0, len(p1))
    crossover_end = random.randint(crossover_start, len(p1))
    child = np.concatenate([p2[0:crossover_start], p1[crossover_start:crossover_end], p2[crossover_end:]])

    return Plane(w=np.array(child[:-1]), b=child[-1])

def crossover_reverse(plane1, plane2):
    """
    Your own crossover function. It should return a single child created by 
    crossing over plane1 and plane2
    """
    
    p1 = np.append(plane1.w, plane1.b)
    p2 = np.append(plane2.w, plane2.b)
    crossover_start = random.randint(0, len(p2))
    crossover_end = random.randint(crossover_start, len(p2))
    child = np.concatenate([p1[0:crossover_start], p2[crossover_start:crossover_end], p1[crossover_end:]])

    return Plane(w=np.array(child[:-1]), b=child[-1])

def crossover_point(plane1, plane2):
    """
    Your other crossover function.
    """
    
    p1 = np.append(plane1.w, plane1.b)
    p2 = np.append(plane2.w, plane2.b)
    crossover_point = random.randint(0, len(p1))
    child = np.concatenate([p1[0:crossover_point], p2[crossover_point:]])

    return Plane(w=np.array(child[:-1]), b=child[-1])

def crossover(plane1, plane2, crossover_type):
    """
    Wrapper function for crossover. Returns the child formed by crossing over 
    plane1 and plane2. 
    crossover_type is a string specifying which crossover function to use.
    """
    if crossover_type == "ordered":
        return crossover_ordered(plane1, plane2)
    # INSERT YOUR OWN CROSSOVER FUNCTION NAMES HERE
    elif crossover_type == "reverse":
        return crossover_reverse(plane1, plane2)
    elif crossover_type == "point":
        return crossover_point(plane1, plane2) 
    
    print("Crossover type invalid")
    return None 

In [ ]:
p1 = Plane(w=np.array([1.0, 2.0, 3.0, 4.0]), b=5.0)
p2 = Plane(w=np.array([11.0, 12.0, 13.0, 14.0]), b=15.0)
crossover_point(p1, p2)

Next, we’ll generalize this to create our offspring population. We will use elitism to retain the best routes from the current population. Then, we use the crossover wrapper function to fill out the rest of the next generation. 

In [ ]:
def breedPopulation(matingpool, eliteSize, crossover_type):
    """
    matingpool: list of individuals selected to form the mating pool
    eliteSize: number of highest ranked individuals preserved in the next generation
    crossover_type: string specifying which crossover function to use
    This function returns the new population created by breeding
    """
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = crossover(pool[i], pool[len(matingpool)-i-1], crossover_type)
        children.append(child)
    return children

## Mutate
Mutation serves an important function in GA, as it helps to avoid local convergence by introducing novel routes that will allow us to explore other parts of the solution space. In this assignment, we shall assume that we mutate a certain fraction of the population, as specified in the variable `mutationRate`.

Note that there can be several possible ways to mutate an individual for this problem. You need to decide on your own how to mutate an individual. One hint could be - change one (some?) of the weights to a random value.

In [ ]:
def mutate(individual, mutationRate):
    """
    individual: type Plane
    This function should mutate a single individual and return the mutated individual.
    Hints given in paragraph above
    """
    
    p1 = np.append(individual.w, individual.b)
    random_value = random.uniform(0,1)
    if random_value < mutationRate: 
        for i in p1:
            p1[random.randint(0, len(p1)-1)] = i * random.uniform(0,10)
        newIndividual = p1
    else:
        newIndividual = p1
    
    return Plane(w=np.array(newIndividual[:-1]), b=newIndividual[-1])

In [ ]:
p1 = Plane(w=np.array([1.0, 2.0, 3.0, 4.0]), b=5.0)
mutate(p1, 0.5)

Next, we can extend the mutate function to run through the new population.  
Please create a function to run mutation over entire population and return the new population.

In [ ]:
def mutatePopulation(population, mutationRate):
    """
    This function should use the above mutate function to mutate each member of the population. Simply iterate over the 
    population and mutate each individual using the mutationRate. It should then return the mutated population
    """
    mutatedPop = []
    for i in population:
        i = mutate(i, mutationRate)
        mutatedPop.append(i)
        
    return mutatedPop

In [ ]:
x = initialPopulation(3)
print(x)
mutatePopulation(x, 0.5)

## Repeat
We’re almost there. Let’s pull these pieces together to create a function that produces a new generation. First, we rank the planes in the current generation using rankPopulation. We then determine our potential parents by running the selection function, which allows us to create the mating pool using the matingPool function. Finally, we then create our new generation using the breedPopulation function (passing in crossover_type) and then applying mutation using the mutatePopulation function.

In [ ]:
def nextGeneration(currentGen, eliteSize, mutationRate, crossover_type, fitness_type):
    """
    This function takes in the current generation, eliteSize and mutationRate and should return the next generation.
    """
    popRanked = rankPopulation(currentGen, fitness_type)
    selectionResults = selection(popRanked, eliteSize)
    pool = matingPool(currentGen, selectionResults)
    children = breedPopulation(pool, eliteSize, crossover_type)
    nextGen = mutatePopulation(children, mutationRate)
    return nextGen

## Final step: Evolution in motion
We finally have all the pieces in place to create our GA! All we need to do is create the initial population, and then we can loop through as many generations as we desire. 

Of course we also want to see the best plane and how much we’ve improved, so we capture the initial training fitness and accuracy, the final distance and accuracy, and the best individual. We should also record the initial and final testing fitness and accuracy. The parameter crossover_type is a string which specifies which crossover function to use.

Before each generation, you can use the fittest individual from the population to evaluate the fitness/accuracy performance of the model on the testing data.

In [ ]:
def geneticAlgorithm(popSize, eliteSize, mutationRate, generations, crossover_type, fitness_type):
    """
    This function should run the genetic algorithm for the specified number of generations
    by following the process outlined  in the "Approach" section given earlier in this notebook. 
    It should print the initial and final training and testing accuracy as well as 
    initial and final fitness.
    It should also generate plots showing the training as well as testing fitness
    with respect to generations.
    """

    population = initialPopulation(popSize)
    
    current_fitness_loc = rankPopulation(population, fitness_type)[0][0]
    current_fitness = population[current_fitness_loc]
    current_train_accuracy = (population[rankPopulation(population, fitness_type)[0][0]].accuracy(X_train, Y_train))
    current_test_accuracy = (population[rankPopulation(population, fitness_type)[0][0]].accuracy(X_test, Y_test))
    
    train_performance = []
    test_performance = []
    
    print('Initial Train Fitness:' + str(rankPopulation(population, fitness_type)[0][1]))
    print('Initial Test Fitness:', current_fitness.fitness(X_test,Y_test, fitness_type))
    print('Initial Train Accuracy:' + str(current_train_accuracy))
    print('Initial Test Accuracy:' + str(current_test_accuracy))
    
    for i in range(0, generations):
        current_fitness = rankPopulation(population, fitness_type)[0][1]
        population = nextGeneration(population, eliteSize, mutationRate, crossover_type, fitness_type)
        train_performance.append(current_fitness)
        test_performance.append(population[rankPopulation(population, fitness_type)[0][0]].fitness(X_test, Y_test, fitness_type))
        
    best_fitness_loc = rankPopulation(population, fitness_type)[0][0]
    best_fitness = population[best_fitness_loc]
    current_train_accuracy1 = (population[rankPopulation(population, fitness_type)[0][0]].accuracy(X_train, Y_train))
    current_test_accuracy1 = (population[rankPopulation(population, fitness_type)[0][0]].accuracy(X_test, Y_test))
        
    print('Final Train Fitness:' + str(rankPopulation(population, fitness_type)[0][1]))
    print('Final Test Fitness:', best_fitness.fitness(X_test,Y_test, fitness_type))
    print('Final Train Accuracy:' + str(current_train_accuracy1))
    print('Final Test Accuracy:' + str(current_test_accuracy1))
    print('Best Plane Index:' + str(rankPopulation(population, fitness_type)[best_fitness_loc]))
    
    plot1 = plt.figure(1)
    plt.plot(train_performance)
    plt.title('Train Performance')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    
    plot2 = plt.figure(2)
    plt.plot(test_performance)
    plt.title('Test Performance')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    
    plt.show()
    
    return best_fitness
    

In [ ]:
# x=[1,2,3,4,5]
# for i in x:
#    geneticAlgorithm(popSize=300, eliteSize=5, mutationRate=0.01, generations=100, crossover_type="point", fitness_type="your_own_fitness")

# Perceptron with Gradient Descent

In this section we will code up the perceptron algorithm with gradient descent.

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

%matplotlib inline

We load the dataset as in previous sections and create the training and testing split.

In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()
print(breast_cancer.data.shape)

In [ ]:
data = pd.DataFrame(breast_cancer.data, columns = breast_cancer.feature_names)
data['class'] = breast_cancer.target

X = data.drop('class', axis = 1)
Y = data['class']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1,stratify= Y, random_state = 1)

X_train = X_train.values
X_test = X_test.values

We create the main Perceptron class and its necessary functions

In [ ]:
class Perceptron:
    def __init__(self):
        self.w = None
        self.b = None

    def model(self, x):
        """
        This function returns the prediction for a single example x
        """
        return (np.dot(x, self.w.T) + self.b >= 0)

    def predict(self, X):
        """ 
        returns the predictions for multiple examples X
        Size of X: (n, dim)
        """
        Y = []
        for x in X:
            res = self.model(x)
            Y.append(res)
        return np.array(Y)

    def fit(self, X, Y, epochs = 500, learning_rate = 0.01):
        """
        This function should train the perceptron by running gradient descent 
        through the entire dataset for the number of epochs specified. The 
        perceptron algorithm can be found in the lecture slides
        """
        self.w = np.ones(X.shape[1])    
        self.b = 0
        for e in range(epochs):
            for i,j in zip(X,Y):
                prediction = self.model(i)
                if prediction == 1 and j == 0:
                    self.w = self.w - (learning_rate * i)
                    self.b = self.b - (learning_rate * 1)
                    
                if prediction == 0 and j == 1:
                    self.w = self.w + (learning_rate * i)
                    self.b = self.b + (learning_rate * 1)

In [ ]:
perceptron = Perceptron()

In [ ]:
perceptron.fit(X_train, Y_train, 1000, 0.1)

Now we generate the predictions of the trained perceptron on the test data, and find the accuracy score when we compare them to the targets.

In [ ]:
Y_pred_test = perceptron.predict(X_test)
print(accuracy_score(Y_pred_test, Y_test))

### Pre-coded perceptron

This section contains a pre-coded perceptron from the SciPy package. Please use this code to generate results for comparison purposes as described in the report instructions.

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()
dataset = breast_cancer.data
targets = breast_cancer.target
print(breast_cancer.data.shape)

X_train, X_test, Y_train, Y_test = train_test_split(dataset, targets, test_size = 0.1, stratify = targets)
print(X_train.shape)

In [ ]:
clf = Perceptron(max_iter=500)
clf.fit(X_train, Y_train)
print(clf.score(X_test, Y_test)) # Get the accuracy score on the test data

# Report and Submission Guidelines

The purpose of this assignment is to compare the performance of the Genetic Algorithm in varying conditions and Perceptron for this problem. 

You need to implement a total of **3** crossover functions and **2** fitness functions, as well as **1** mutation function. One fitness function has already been written for you, and the other can be found in Lecture 3. As for mutation, you can choose any mutation rate or even a different mutation strategy which gives you the best testing performance.

The best way to compare different algorithms is to run many iterations, and record the results. For this assignment, each time you call the geneticAlgorithm function, you use a certain crossover and fitness function. There are 3 crossover functions and 2 fitness function, which means there are 6 possible combinations. We want to compare the performance of these different combinations with each other as well as with Gradient Descent.

For each combination (ie. crossover_ordered paired with fitness1, crossover_ordered paired with fitness2) you should run the geneticAlgorithm function 10 times. Then, take the average final **testing** accuracy of these 10 iterations for this specific combination, and report these in a table. Also add the accuracies of both your coded as well as the Scipy perceptron algorithm to this table.

The final report should contain an accuracy table, that has the average final testing accuracies for each combination of the genetic algorithm. Additionally, for each combination, please produce one plot (pick one of the 10 iterations) that plots fitness versus number of generations for both the training and testing data.

To summarize, you need to write the following in your report:
- Accuracies table as specified above
- Training and testing fitness plots with respect to generation
- Description of your mutation function as well as crossover functions
- Brief (1 paragraph) description of why your mutation/crossover functions work well, or why they don't work well. Basically a possible justification of the performance.